In [130]:
from typing import NamedTuple

norcal_ports = ["SFO", "OAK", "SJC"]
socal_ports = ["LAX", "SNA", "SAN"]
japan_ports = [
    "HND",
    "FUK",
    "CTS",
    "OKA",
    "ITM",
    "NRT",
    "KIX",
    "NGO",
    "KOJ",
    "UKB",
    "SDJ",
    "ISG",
    "KMI",
    "KMJ",
    "NGS",
    "MYJ",
]

prefered_japan_ports = ["HND", "FUK", "KIX", "NGO", "CTS", "OKA"]

start_dates = ["2024-03-21", "2024-03-22", "2024-03-23"]
end_date = ["2024-03-30", "2024-03-31", "2024-04-01"]


class MultiCity(NamedTuple):
    home_out: str
    dest_in: str
    dest_out: str
    home_in: str


def generate_open_jaw(us_ports, japan_ports) -> list[MultiCity]:
    return [
        MultiCity(u_out, j_in, j_out, u_in)
        for u_out in us_ports
        for j_in in japan_ports
        for j_out in japan_ports
        for u_in in us_ports
    ]


In [131]:
import requests
import json
import config
from datetime import date


def to_ddmmyy(d: date) -> str:
    return d.strftime("%d/%m/%Y")


def flight_search(ports: MultiCity, date_from: date, date_to: date) -> list[dict]:
    '''Returns the top 5 flight plans json responses from kiwi'''
    url = "https://api.tequila.kiwi.com/v2/flights_multi"

    payload = {
        "requests": [
            {
                "limit": 5,
                "sort": "quality",
                "curr": "USD",
                "fly_to": ports.dest_in,
                "fly_from": ports.home_out,
                "date_from": to_ddmmyy(date_from),
                "date_to": to_ddmmyy(date_from),
                "adults": 1,
            },
            {
                "limit": 5,
                "sort": "quality",
                "curr": "USD",
                "fly_to": ports.home_in,
                "fly_from": ports.dest_out,
                "date_from": to_ddmmyy(date_to),
                "date_to": to_ddmmyy(date_to),
                "adults": 1,
            },
        ],
    }

    json_payload = json.dumps(payload)

    headers = {
        "Content-Type": "application/json",
        "apikey": config.kiwi_multi_city_key,
    }

    try:
        response = requests.post(url, data=json_payload, headers=headers)
        response.raise_for_status()  # Raise an exception if the response status code is not 2xx

        response_data = json.loads(response.text)

        return response_data

    except Exception as e:
        print("Error: ", e)
    return None

In [132]:
class RawSearchResults(NamedTuple):
    airports: MultiCity
    date_from: date
    date_to: date
    response: dict

In [133]:
class FlightInfo(NamedTuple):
    airports: MultiCity
    date_from: date
    date_to: date
    price: int
    duration: float
    quality: float
    link: str


def extract_flight_info(itinerary: dict):
    price: int = itinerary["price"]
    duration: float = round((itinerary["duration"]) / 3600, 2)
    quality: float = itinerary["quality"]
    link: str = itinerary["deep_link"]


    return price, duration, quality, link


def get_best_flights(
    ports: MultiCity, date_from: date, date_to: date
) -> list[FlightInfo]:

    try: 
        raw_top_5_itineraries = flight_search(ports, date_from, date_to)

        best_flights = [
            FlightInfo(ports, date_from, date_to, *extract_flight_info(i))
            for i in raw_top_5_itineraries
        ]


        return best_flights

    except Exception as e:
        print("Error: ", e)

    return None

In [134]:
ports = MultiCity("LAX", "NRT", "NRT", "LAX")
date_from = date(2024, 3, 22)
date_to = date(2024, 4, 1)

results = get_best_flights(ports, date_from, date_to)
results

[FlightInfo(airports=MultiCity(home_out='LAX', dest_in='NRT', dest_out='NRT', home_in='LAX'), date_from=datetime.date(2024, 3, 22), date_to=datetime.date(2024, 4, 1), price=1186, duration=28.58, quality=1278.86, link='https://www.kiwi.com/deep?affilid=at527japanmulti&currency=USD&flightsId=243e209f4d5c00006b08ffba_0%7C209f243e4d66000066f8049c_0%7C209f243e4d66000066f8049c_1&from=LAX&lang=en&passengers=1&to=NRT&type2=multicity&booking_token=GWkTHCn_6ISmkncu7QZFE3if9pjcJP-5cZjlvVQgBcRbZchpnAMbdDmau1kDRd3vZyjS2rkM8aKU2XTpqEshBfVDNQHwDtFj3iuFJArcMBmoQ5Crh-A9VJkxmozr6uG7xSgtk593Azq6PliuYA2qdOPssPwj0TM8ils3kNpRvotl0vhEktniaXAbncM87vzfPoigv7o1LnBHxXiRhzME4AHAUKJmcP5LgAlxzf9ZECRUVzVPIRMKW6EUm92eYig5NA1v8qYdEdyvnxhO9hEsLztFB3yCI-RS7Gmz67hXhAiWl_zEziYXwAquqILT8vdo6LNIlMIgSJEqFQ1-bzHOKqAaSk84y0IMoczha1dRepumqSH8FWS17-l0cTbdIR5OJVCJ05cQBBLPAG90_6Z5wl88YlMckus6qYm0DA8Tgivi37Db7On323x14PfpbV-Pa0vuGCvXhCxl0trFw2PX1Dkg-awTWZc5z-Mjdxc-6MoVm_81FX2anD_kH2kNWSp5HCbKe5YScg61rDa1V2-Vo7w=='),
 FlightInfo(airp

In [140]:
import concurrent.futures
import time

multi_city_flights = generate_open_jaw(["LAX"], prefered_japan_ports)

def search_all_multicity(multi_city_flights, date_from, date_to):
    all_results = []
    for f in multi_city_flights:
        print("Searching: " + str(f))
        all_results += get_best_flights(f, date_from, date_to)

    return all_results


# def search_all_multicity(multi_city_flights, date_from, date_to):
#     all_results = []

#     # Create a ThreadPoolExecutor for concurrent execution
#     with concurrent.futures.ThreadPoolExecutor() as executor:
#         # Use executor.map to run get_best_flights concurrently for each flight
#         futures = [
#             executor.submit(get_best_flights, f, date_from, date_to)
#             for f in multi_city_flights
#         ]

#         # Retrieve results as they complete
#         for future in concurrent.futures.as_completed(futures):
#             time.sleep(3)
#             best = future.result()
#             if (best is None):
#                 continue
#             all_results += best
#             print(best)

#     return all_results

In [141]:
all_best_flights = search_all_multicity(multi_city_flights, date_from, date_to)

Searching: MultiCity(home_out='LAX', dest_in='HND', dest_out='HND', home_in='LAX')
Searching: MultiCity(home_out='LAX', dest_in='HND', dest_out='FUK', home_in='LAX')
Searching: MultiCity(home_out='LAX', dest_in='HND', dest_out='KIX', home_in='LAX')
Searching: MultiCity(home_out='LAX', dest_in='HND', dest_out='NGO', home_in='LAX')
Searching: MultiCity(home_out='LAX', dest_in='HND', dest_out='CTS', home_in='LAX')
Searching: MultiCity(home_out='LAX', dest_in='HND', dest_out='OKA', home_in='LAX')
Searching: MultiCity(home_out='LAX', dest_in='FUK', dest_out='HND', home_in='LAX')
Searching: MultiCity(home_out='LAX', dest_in='FUK', dest_out='FUK', home_in='LAX')
Searching: MultiCity(home_out='LAX', dest_in='FUK', dest_out='KIX', home_in='LAX')
Searching: MultiCity(home_out='LAX', dest_in='FUK', dest_out='NGO', home_in='LAX')
Searching: MultiCity(home_out='LAX', dest_in='FUK', dest_out='CTS', home_in='LAX')
Searching: MultiCity(home_out='LAX', dest_in='FUK', dest_out='OKA', home_in='LAX')
Sear

In [142]:
len(all_best_flights)

180

In [143]:
import csv

socal_csv = "socal_flights.csv"

# Define the CSV file's header
csv_header = [
    "Home Out",
    "Dest In",
    "Dest Out",
    "Home In",
    "Date From",
    "Date To",
    "Price",
    "Duration",
    "Quality",
    "Link"
]

# Write FlightInfo objects to the CSV file
with open(socal_csv, mode="w", newline="") as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(csv_header)

    # Use a list comprehension to generate rows
    rows = [
        [
            flight_info.airports.home_out,
            flight_info.airports.dest_in,
            flight_info.airports.dest_out,
            flight_info.airports.home_in,
            flight_info.date_from,
            flight_info.date_to,
            flight_info.price,
            flight_info.duration,
            flight_info.quality,
            flight_info.link,
        ]
        for flight_info in all_best_flights
    ]

    # Write the generated rows to the CSV file
    writer.writerows(rows)

print(f"Flight info has been written to {socal_csv}")

Flight info has been written to socal_flights.csv
